In [1]:
import os
import sys
import pandas as pd
import requests
from igdb.wrapper import IGDBWrapper
import json
import matplotlib.pyplot as plt
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess_1_cleaning import *

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
GRANT_TYPE = os.getenv("GRANT_TYPE")

# Importing and segmenting the data

In [2]:
data = pd.read_csv('../raw_data/all_game_data_v1_corrected2', low_memory=False)

In [3]:
df = cleaning_in_notebook(data)

#Data with no reviews

#Creating a mask
df_mask = df['avg_review'] == 0
df_no_reviews = df[df_mask]

#Sorting and removing upcoming games to reduce load
df_sorted = df_no_reviews.sort_values('release_date')
time_mask = df_sorted['release_date'] < "2023-08-01"
no_duplicates_recent_games = df_sorted[time_mask].drop_duplicates('title')
list_of_titles_without_review = no_duplicates_recent_games['title']


#Data with reviews
df_good_mask = df['avg_review'] != 0
df_with_reviews = df[df_good_mask]
with_reviews_dropped_dupes = df_with_reviews.drop_duplicates('title')
titles_with_reviews = with_reviews_dropped_dupes['title']

/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/Users/antonis/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [48]:
api_data = pd.read_csv('../raw_data/api_on_data_without_reviews')

In [42]:
def remove_link_trail(string):
    update = string.replace('https://www.backloggd.com', '')
    return update

In [46]:
no_duplicates_recent_games['url'] = no_duplicates_recent_games['url'].apply(remove_link_trail)

In [49]:
smaller_df = no_duplicates_recent_games[['title', 'avg_review', 'url']]

In [59]:
smaller_df.drop_duplicates()

,title,avg_review,url
116719,Nim,0.0,/games/nim/
116714,Mouse in the Maze,0.0,/games/mouse-in-the-maze/
116708,Indy 500,0.0,/games/indy-500--3/
116705,Wild West,0.0,/games/wild-west--1/
116704,Space Travel,0.0,/games/space-travel/
...,...,...,...
1561,Enjoy the Diner,0.0,/games/enjoy-the-diner/
1562,The 4 Masters of Melody Ex,0.0,/games/the-4-masters-of-melody-ex/
1563,Super Fighting Jam,0.0,/games/super-fighting-jam/
1564,Battle of Tarlis,0.0,/games/battle-of-tarlis/


In [73]:
game_ids = pd.read_csv('../raw_data/all_links_with_game_id_v1').drop_duplicates()
game_ids

,link,game_id
0,/games/pilotxross/,123757
1,/games/plague-hunter/,106694
2,/games/the-god-slayer/,259874
3,/games/rise-of-rebellion/,211568
4,/games/penny-blood/,214912
...,...,...
120091,/games/play-ball/,40401
120092,/games/baseball--3/,11412
120093,/games/the-oregon-trail/,11325
120094,/games/computer-space/,11245


In [69]:
to_call = smaller_df.merge(game_ids, left_on='url', right_on='link').drop_duplicates('title')
to_call

,title,avg_review,url,link,game_id
0,Volleyball,0.0,/games/volleyball--2/,/games/volleyball--2/,109446
1,Moonlander,0.0,/games/lunar-lander--1/,/games/lunar-lander--1/,9129
2,Paddle Battle,0.0,/games/paddle-battle--1/,/games/paddle-battle--1/,172781
3,Volly,0.0,/games/volly/,/games/volly/,130859
4,Empire,0.0,/games/empire--1/,/games/empire--1/,11518
...,...,...,...,...,...
62223,Enjoy the Diner,0.0,/games/enjoy-the-diner/,/games/enjoy-the-diner/,250232
62224,The 4 Masters of Melody Ex,0.0,/games/the-4-masters-of-melody-ex/,/games/the-4-masters-of-melody-ex/,259774
62225,Super Fighting Jam,0.0,/games/super-fighting-jam/,/games/super-fighting-jam/,257686
62226,Battle of Tarlis,0.0,/games/battle-of-tarlis/,/games/battle-of-tarlis/,259587


In [71]:
to_call.to_csv('../raw_data/no_reviews_to_fetch_from_api', index=False)

# Creating a function that cleans the urls for easy merging

Mergine will be with game_ids. 

game_id's will be used in the api to achieve a lower failure rate

In [18]:
#Removes the first part of the url
def clean_urls(url):
    shorter_url = url[25:]
    
    return shorter_url

In [24]:
#URL should be added back into the dataframe
no_duplicates_recent_games['url'] = no_duplicates_recent_games['url'].apply(clean_urls)

# Merging game_id

In [25]:
df_with_game_id = pd.read_csv('../raw_data/all_links_test2')

In [26]:
df_with_game_id

,link,game_id
0,/games/pilotxross/,123757
1,/games/plague-hunter/,106694
2,/games/the-god-slayer/,259874
3,/games/rise-of-rebellion/,211568
4,/games/penny-blood/,214912
...,...,...
355,/games/ashes-of-creation/,27420
356,/games/reclaim-the-sea/,216954
357,/games/interwoven/,204970
358,/games/loco-motive/,141805


In [27]:
df_with_game_id.merge(no_duplicates_recent_games, left_on='link', right_on='url')

,link,game_id,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url


In [31]:
x_cross = no_duplicates_recent_games['title'] == 'airportsim'

no_duplicates_recent_games[x_cross]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url


# Merging API data without reviews

In [32]:
api_data = pd.read_csv('../raw_data/api_on_data_without_reviews', low_memory=False)

In [38]:
api_mask = api_data['aggregated_rating'].isna()
api_data_with_reviews = api_data[~api_mask]

In [39]:
api_data_with_reviews.merge(no_duplicates_recent_games, on='title')

,id,aggregated_rating,aggregated_rating_count,game_engines,game_modes,player_perspectives,themes,rating,title,release_date,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url
0,192153,75.000000,1.0,NaN,Single player,NaN,Mystery,NaN,Interceptor,19760301,...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,
1,15507,62.000000,1.0,NaN,NaN,NaN,NaN,NaN,PBA Bowling,19810831,...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,
2,22848,42.000000,1.0,NaN,Single player,Bird view / Isometric,Action,NaN,Eliminator,19811201,...,0,2,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,
3,4,63.642857,17.0,Unreal Engine 3,Single player,First person,Sandbox,70.566355,Thief,19811231,...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,
4,2753,80.000000,1.0,NaN,Multiplayer,Bird view / Isometric,Science fiction,79.641885,Zaxxon,19821231,...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,198228,73.000000,1.0,NaN,Co-operative,Virtual Reality,Action,NaN,Espire 2,20221117,...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,
1440,216586,80.000000,1.0,NaN,Co-operative,Side view,Action,NaN,Toaplan Arcade 1,20221215,...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,
1441,216120,67.000000,1.0,NaN,Single player,Side view,Fantasy,NaN,Alphadia Neo,20221216,...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,
1442,155706,70.333333,4.0,NaN,Single player,Bird view / Isometric,Fantasy,33.940084,Potion Permit,20221231,...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,
